In [65]:

from backtesting import Backtest, Strategy
from backtesting.test import GOOG, SMA
from backtesting.lib import crossover

import yfinance as yf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

tf.random.set_seed(42)
np.random.seed(42)

In [41]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 2)
        self.ma2 = self.I(SMA, price, 5)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(GOOG, SmaCross, commission=0.002, exclusive_orders=True)
stats = bt.run()
bt.plot()

Backtest.run:   0%|          | 0/2143 [00:00<?, ?bar/s]

GridPlot(id='p3801', ...)

In [10]:
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    94.27374
Equity Final [$]                  56263.51934
Equity Peak [$]                   56309.05934
Commissions [$]                   10563.95154
Return [%]                          462.63519
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                    22.46598
Volatility (Ann.) [%]                 37.4129
CAGR [%]                             14.99343
Sharpe Ratio                          0.60049
Sortino Ratio                          1.1445
Calmar Ratio                           0.6621
Alpha [%]                           450.62135
Beta                                  0.01978
Max. Drawdown [%]                   -33.93159
Avg. Drawdown [%]                    -6.16072
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                          

In [22]:
hist_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2023-04-12 00:00:00-04:00,28.408978,28.494690,28.228028,28.370884,9800,0.0,0.0,0.0
2023-04-13 00:00:00-04:00,28.532785,28.656593,28.508976,28.656593,9900,0.0,0.0,0.0
2023-04-14 00:00:00-04:00,28.656593,28.656593,28.447074,28.627069,5100,0.0,0.0,0.0
2023-04-17 00:00:00-04:00,28.513738,28.513738,28.295647,28.399454,7200,0.0,0.0,0.0
2023-04-18 00:00:00-04:00,28.599451,28.970872,28.581355,28.637545,8400,0.0,0.0,0.0


In [59]:
symbol = "DAX"
ticker = yf.Ticker(symbol)

hist_data = ticker.history(period="2y")

# Create train-test split based on specific dates
train_cutoff_date = "2025-01-02"
test_end_date = "2025-03-31"

# Split the original historical data
train_hist_data = hist_data[hist_data.index < train_cutoff_date]
test_hist_data = hist_data[(hist_data.index >= train_cutoff_date) & 
                           (hist_data.index <= test_end_date)]

# Prepare the data for LSTM
train_data = train_hist_data[["Close"]].copy().reset_index()
test_data = test_hist_data[["Close"]].copy().reset_index()

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data[["Close"]])
test_scaled = scaler.transform(test_data[["Close"]])


In [83]:
# Improved LSTM model parameters
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 20
X_train, y_train = create_sequences(train_scaled, seq_length)
X_test, y_test = create_sequences(test_scaled, seq_length)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(seq_length, 1), recurrent_regularizer=l2(0.01)),
    Dropout(0.3),
    LSTM(64, recurrent_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

# Use a fixed learning rate instead of a scheduler
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='huber')

early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, min_delta=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


Epoch 1/150


/home/kuba/projects/ekonomia/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 1.2172 - val_loss: 0.8011 - learning_rate: 0.0010
Epoch 2/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6869 - val_loss: 0.4544 - learning_rate: 0.0010
Epoch 3/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4008 - val_loss: 0.2665 - learning_rate: 0.0010
Epoch 4/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2368 - val_loss: 0.1580 - learning_rate: 0.0010
Epoch 5/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1419 - val_loss: 0.0974 - learning_rate: 0.0010
Epoch 6/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0858 - val_loss: 0.0591 - learning_rate: 0.0010
Epoch 7/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0523 - val_loss: 0.0353 - learning_rate: 0.0010
Epoch 8/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0325 - val_loss: 0.0271 - learning_rate: 0.0010
Epoch 9/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0225 - val_loss: 0.0155 - learning_rate: 0.0010
Epoch 10/150
21/21 ━━━

In [84]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions back to original scale
train_predict = scaler.inverse_transform(train_predict)
y_train_inv = scaler.inverse_transform(y_train)
test_predict = scaler.inverse_transform(test_predict)
y_test_inv = scaler.inverse_transform(y_test)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train_inv, train_predict))
test_rmse = np.sqrt(mean_squared_error(y_test_inv, test_predict))
train_mape = np.mean(np.abs((y_train_inv - train_predict) / y_train_inv)) * 100
test_mape = np.mean(np.abs((y_test_inv - test_predict) / y_test_inv)) * 100

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")
print(f"Train MAPE: {train_mape:.2f}%")
print(f"Test MAPE: {test_mape:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Train RMSE: 0.56
Test RMSE: 0.98
Train MAPE: 1.51%
Test MAPE: 1.95%


In [116]:
class LSTMOptionStrategy(Strategy):
    def init(self):
        price = self.data.Close
        self.sma20 = self.I(SMA, price, 20)
        self.model = model
        self.scaler = scaler
        self.predictions = self.I(lambda: np.full(len(self.data), np.nan))
        self.buy_signals = self.I(lambda: np.zeros(len(self.data)))
        self.sell_signals = self.I(lambda: np.zeros(len(self.data)))
        
        # Thresholds
        self.tomorrow_threshold = 0.3  # % increase for tomorrow's prediction
        self.week_avg_threshold = 0.5  # % increase for week's avg
        self.days20_avg_threshold = 0.8  # % increase for 20 days avg
        
        self.tomorrow_sell_threshold = -0.2  # % decrease for tomorrow's prediction
        self.week_avg_sell_threshold = -0.4  # % decrease for week's avg
        self.days20_avg_sell_threshold = -0.6  # % decrease for 20 days avg
    
    def get_prediction(self, days_ahead=5):
        if np.isnan(self.sma20[-1]):
            return None
        
        recent_prices = np.array([self.data.Close[-i] for i in range(20, 0, -1)])
        recent_df = pd.DataFrame(recent_prices, columns=["Close"])
        scaled_data = self.scaler.transform(recent_df)
        
        # Make predictions for each day in the week ahead
        future_predictions = []
        temp_data = scaled_data.copy()
        
        for _ in range(days_ahead):
            X = temp_data[-20:].reshape(1, 20, 1)
            next_pred = self.model.predict(X, verbose=0)[0, 0]
            future_predictions.append(next_pred)
            temp_data = np.vstack([temp_data, next_pred])  # Append and loop
        
        # Convert scaled predictions back to original scale
        future_df = pd.DataFrame(np.array(future_predictions).reshape(-1, 1), columns=["Close"])
        future_prices = self.scaler.inverse_transform(future_df)
        
        # Get predictions for future 20 days
        future_20_predictions = []
        if days_ahead < 20:
            days_to_predict = 20 - days_ahead
            for _ in range(days_to_predict):
                X = temp_data[-20:].reshape(1, 20, 1)
                next_pred = self.model.predict(X, verbose=0)[0, 0]
                future_20_predictions.append(next_pred)
                temp_data = np.vstack([temp_data, next_pred])  # Append and loop
                
        future_20_df = pd.DataFrame(np.array(future_20_predictions).reshape(-1, 1), columns=["Close"])
        future_20_prices = self.scaler.inverse_transform(future_20_df)  # Scale conversion
        future_20_prices = np.append(future_prices.flatten(), future_20_prices.flatten())
        
        return future_prices.flatten(), future_20_prices  # returns both
    
    def analyze_trend(self, predictions, future_20_prices):
        current_price = self.data.Close[-1]
        
        tomorrow_change = (predictions[0] - current_price) / current_price * 100 # % Tomorrow
        week_avg_change = (sum(predictions) / len(predictions) - current_price) / current_price * 100 # % Week
        days20_avg_change = (sum(future_20_prices) / len(future_20_prices) - current_price) / current_price * 100 # % 20 Days
        
        return {
            'tomorrow_change': tomorrow_change,
            'week_avg_change': week_avg_change,
            'days20_avg_change': days20_avg_change
        }
    
    def next(self):
        if np.isnan(self.sma20[-1]):
            return
            
        current_price = self.data.Close[-1]
        preds, future_20_prices = self.get_prediction(days_ahead=5) # Correct calls
        
        if preds is None:
            return
            
        trend = self.analyze_trend(preds, future_20_prices) # Both params passed in
        
        # Buy conditions
        if (trend['tomorrow_change'] > self.tomorrow_threshold or
            trend['week_avg_change'] > self.week_avg_threshold or
            trend['days20_avg_change'] > self.days20_avg_threshold):
            
            self.buy_signals[-1] = 1
            self.buy()
            print(f"Executed BUY at {current_price:.2f} - Conditions Met")
                
        # Sell Conditions
        elif (trend['tomorrow_change'] < self.tomorrow_sell_threshold or
              trend['week_avg_change'] < self.week_avg_sell_threshold or
              trend['days20_avg_change'] < self.days20_avg_sell_threshold):
            
            self.sell_signals[-1] = 1
            self.sell()
            print(f"Executed SELL at {current_price:.2f} - Conditions Met")


In [117]:
bt = Backtest(test_hist_data, LSTMOptionStrategy, commission=0.002, exclusive_orders=True)
stats = bt.run()
bt.plot()

Backtest.run:   0%|          | 0/40 [00:00<?, ?bar/s]

Executed BUY at 35.71 - Conditions Met
Executed SELL at 36.51 - Conditions Met
Executed SELL at 36.93 - Conditions Met
Executed SELL at 36.46 - Conditions Met
Executed SELL at 36.77 - Conditions Met
Executed SELL at 37.15 - Conditions Met
Executed SELL at 37.62 - Conditions Met
Executed SELL at 38.22 - Conditions Met
Executed SELL at 38.35 - Conditions Met
Executed SELL at 38.80 - Conditions Met
Executed SELL at 37.96 - Conditions Met
Executed SELL at 38.05 - Conditions Met
Executed BUY at 37.64 - Conditions Met
Executed SELL at 38.47 - Conditions Met
Executed SELL at 38.46 - Conditions Met
Executed BUY at 37.78 - Conditions Met
Executed SELL at 39.17 - Conditions Met
Executed SELL at 39.08 - Conditions Met
Executed SELL at 40.72 - Conditions Met
Executed SELL at 40.64 - Conditions Met
Executed SELL at 40.89 - Conditions Met
Executed SELL at 39.70 - Conditions Met
Executed SELL at 39.75 - Conditions Met
Executed SELL at 40.06 - Conditions Met
Executed SELL at 40.74 - Conditions Met
Exe

/home/kuba/projects/ekonomia/.venv/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p15428', ...)

In [81]:
stats

Start                     2025-01-02 00:00...
End                       2025-03-31 00:00...
Duration                     87 days 23:00:00
Exposure Time [%]                        90.0
Equity Final [$]                   8646.10485
Equity Peak [$]                   10356.16918
Commissions [$]                    1961.45123
Return [%]                          -13.53895
Buy & Hold Return [%]                16.47304
Return (Ann.) [%]                   -45.71922
Volatility (Ann.) [%]                12.48014
CAGR [%]                            -34.08395
Sharpe Ratio                         -3.66336
Sortino Ratio                        -2.51895
Calmar Ratio                         -2.59316
Alpha [%]                            -7.66105
Beta                                 -0.35682
Max. Drawdown [%]                   -17.63068
Avg. Drawdown [%]                   -17.63068
Max. Drawdown Duration       75 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

In [42]:
bt = Backtest(test_hist_data, SmaCross, commission=0.002, exclusive_orders=True)
stats = bt.run()
bt.plot()

Backtest.run:   0%|          | 0/55 [00:00<?, ?bar/s]

/home/kuba/projects/ekonomia/.venv/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p4177', ...)